In [ ]:
import os
import faiss
import numpy as np
from tqdm import tqdm
from sentence_transformers import SentenceTransformer

## Этап 1. Загрузка текстов из файлов

In [ ]:
TEXT_FILES_DIR = "garant"

In [ ]:
def load_texts_from_folder(folder_path):
    texts = []
    filenames = []
    for file in tqdm(sorted(os.listdir(folder_path)), desc="Загрузка текстов"):
        if file.endswith(".txt"):
            file_path = os.path.join(folder_path, file)
            with open(file_path, "r", encoding="utf-8") as f:
                text = f.read().strip()
                texts.append(text)
                filenames.append(file)
    return texts, filenames

In [ ]:
texts, filenames = load_texts_from_folder(TEXT_FILES_DIR)

## Этап 2. Создание эмбеддингов

In [ ]:
MODEL_NAME = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
model = SentenceTransformer(MODEL_NAME)

In [ ]:
text_embeddings = model.encode(texts, batch_size=16, convert_to_numpy=True, show_progress_bar=True)
print(f"\nРазмерность эмбеддингов: {text_embeddings.shape}")

## Этап 3: Создание FAISS-индекса

FAISS используется для быстрого поиска среди эмбеддингов

In [ ]:
norms = np.linalg.norm(text_embeddings, axis=1, keepdims=True)
text_embeddings_normalized = text_embeddings / norms
dimension = text_embeddings.shape[1]

Создаём FAISS-индекс

In [ ]:
index = faiss.IndexFlatL2(dimension)
index.add(text_embeddings_normalized.astype('float32'))

Сохраняем индекс для последующего использования

In [ ]:
faiss.write_index(index, "results_MPNet_Default/text_index.faiss")
np.save("results_MPNet_Default/filenames.npy", np.array(filenames))  # сохраняем список имён файлов

## Этап 4: Функция поиска похожих текстов

In [ ]:
def find_similar_texts(query, top_k=5):
    query_embedding = model.encode([query], convert_to_numpy=True) # преобразуем запрос в эмбеддинг
    distances, indices = index.search(query_embedding, top_k) # выполняем поиск в FAISS

    print("Наиболее похожие тексты:")
    for i, idx in enumerate(indices[0]):
        print(f"{i+1}. {filenames[idx]} (дистанция: {distances[0][i]:.4f})")
        print(texts[idx][:300] + "...")  # выводим первые 300 символов текста
        print("-" * 80)

In [ ]:
query_text = "Международный день инвалидов в Хабаровске"
find_similar_texts(query_text, top_k=20)